In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import re
from tqdm import tqdm,trange
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.chdir('C:\\Users\\ASUS\\PycharmProjects\\RahnemaProject')

In [3]:
df = pd.read_csv('Feature_extracted.csv')

In [4]:
method = pd.get_dummies(df['method'],drop_first=True)
df = pd.concat([df,method],axis=1)

In [5]:
from sklearn.preprocessing import LabelEncoder
geole = LabelEncoder()

In [6]:
df.loc[df['last_page'].isnull()==True,'last_page']= 'NaN'

le = LabelEncoder()
df['Le_coded_path']=le.fit_transform(df['coded_path'])
df['le_last_page'] = df.groupby(['client_ip','user_agent'])['Le_coded_path'].shift(1)

In [7]:
df.loc[df['le_last_page'].isnull()==True,'le_last_page']= df['le_last_page'].max()+1

In [8]:
df['last_refer'] = df.loc[df['last_refer'].isnull()==True,'last_refer']= 0

In [9]:
df.loc[df['geolocation'].isnull()==True,'geolocation']= 'NaN'

In [10]:
df['geo_coded'] = geole.fit_transform(df['geolocation'])

In [11]:
userLe = LabelEncoder()
df['user_coded'] = userLe.fit_transform(df['user_agent'])

In [12]:
ipLe = LabelEncoder()

df['ip_encoded'] = ipLe.fit_transform(df['client_ip'])

In [13]:
df.loc[df['response_time'].isnull()==True,'response_time']=0


In [123]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260035 entries, 0 to 1260034
Data columns (total 21 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   client_ip        1260035 non-null  object 
 1   date_time        1260035 non-null  object 
 2   status_code      1260035 non-null  int64  
 3   response_length  1260035 non-null  int64  
 4   user_agent       1260035 non-null  object 
 5   response_time    1260035 non-null  float64
 6   method           1260035 non-null  object 
 7   path             1260035 non-null  object 
 8   geolocation      1260035 non-null  object 
 9   last_refer       1260035 non-null  int64  
 10  coded_path       1260035 non-null  object 
 11  last_page        1260035 non-null  object 
 12  Head             1260035 non-null  uint8  
 13  Options          1260035 non-null  uint8  
 14  Post             1260035 non-null  uint8  
 15  Put              1260035 non-null  uint8  
 16  Le_coded_path    1

In [144]:
date_time = pd.to_datetime(df['date_time']).astype(int)/10**6
last_date_time = pd.to_datetime(df['date_time']).astype(int)/10**6

In [146]:
model_df = df[['user_coded','ip_encoded','status_code','response_length','response_time','Head'
               ,'Options','Post','Put','geo_coded','Le_coded_path','le_last_page']]

In [148]:
model_df = pd.concat([model_df,date_time,last_date_time],axis=1)

In [149]:
model_df.to_csv('processed_df.csv',index=False)

---

---

In [ ]:
df[(df['response_length']>0.4*10**4)&(df['response_time']>10**7)]

In [ ]:
df[df['client_ip']=='244.3.200.14']

In [9]:
df.loc[df['response_time'].argmax()]

client_ip                                              78.157.60.161
date_time                                  2021-05-12 12:41:12+04:30
status_code                                                      101
response_length                                               248408
user_agent         Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...
response_time                                            1.75047e+08
method                                                           Get
path                                       /api/v2/connect/806030186
geolocation                                                     Iran
last_refer                                 2021-05-12 11:46:43+04:30
coded_path                                            12-18-22-37009
last_page                                             12-18-22-27653
Name: 774051, dtype: object

In [ ]:
df[df['client_ip']=='78.157.60.161']

In [10]:
df.loc[df['response_length'].argmax()]



client_ip                                              14.226.145.71
date_time                                  2021-05-12 13:09:15+04:30
status_code                                                      200
response_length                                            102233543
user_agent         Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
response_time                                                   6328
method                                                           Get
path                /cdn/downloads/videos/workshops/cat/session3.mp4
geolocation                                                  Vietnam
last_refer                                 2021-05-12 13:09:02+04:30
coded_path                                      1-887-92-41632-41633
last_page                                                          4
Name: 899972, dtype: object

In [ ]:
df[df['client_ip']=='14.226.145.71']

In [ ]:
df.sort_values('response_time',ascending=False).head()

In [20]:
list_index = df.sort_values('response_length',ascending=False).head().index

In [31]:
new_df = df.drop(list(list_index))